Current and near-term quantum computers suffer from imperfections, as we repeatedly pointed it out. This is why we cannot run long algorithms, that is, deep circuits on them. A new breed of algorithms started to appear since 2013 that focus on getting an advantage from imperfect quantum computers. The basic idea is extremely simple: run a short sequence of gates where some gates are parametrized. Then read out the result, make adjustments to the parameters on a classical computer, and repeat the calculation with the new parameters on the quantum hardware. This way we create an iterative loop between the quantum and the classical processing units, creating classical-quantum hybrid algorithms.

<img src="../figures/hybrid_classical_quantum.svg" alt="Hybrid classical-quantum paradigm" style="width: 400px;"/>

These algorithms are also called variational to reflect the variational approach to changing the parameters. One of the most important example of this approach is the quantum approximate optimization algorithm, which is the subject of this notebook.

# Quantum approximate optimization algorithm

The quantum approximate optimization algorithm (QAOA) is shallow-circuit variational algorithm for gate-model quantum computers that was inspired by quantum annealing. We discretize the adiabatic pathway in some $p$ steps, where $p$ influences precision. Each discrete time step $i$ has two parameters, $\beta_i, \gamma_i$. The classical variational algorithms does an optimization over these parameters based on the observed energy at the end of a run on the quantum hardware.

More formally, we want to discretize the time-dependent $H(t)=(1-t)H_0 + tH_1$ under adiabatic conditions. We achieve this by Trotterizing the unitary. For instance, for time step $t_0$, we can split this unitary as $U(t_0) = U(H_0, \beta_0)U(H_1, \gamma_0)$. We can continue doing this for subsequent time steps, eventually splitting up the evolution to $p$ such chunks:

$$
U = U(H_0, \beta_0)U(H_1, \gamma_0)\ldots U(H_0, \beta_p)U(H_1, \gamma_p).
$$

At the end of optimizing the parameters, this discretized evolution will approximate the adiabatic pathway:

<img src="../figures/qaoa_process.svg" alt="Quantum approximate optimization algorithm" style="width: 400px;"/>

The Hamiltonian $H_0$ is often referred to as the driving or mixing Hamiltonian, and $H_1$ as the cost Hamiltonian. The simplest mixing Hamiltonian is $H_0 = -\sum_i \sigma^X_i$, the same as the initial Hamiltonian in quantum annealing. By alternating between the two Hamiltonian, the mixing Hamiltonian drives the state towards and equal superposition, whereas the cost Hamiltonian tries to seek its own ground state.

Let us import the necessary packages first:

In [1]:
import numpy as np
from functools import partial
from pyquil import Program, api
from pyquil.paulis import PauliSum, PauliTerm, exponential_map, sZ
from pyquil.gates import *
from scipy.optimize import minimize
from forest_tools import *
np.set_printoptions(precision=3, suppress=True)
qvm_server, quilc_server, fc = init_qvm_and_quilc()

In [2]:
n_qubits = 2

Now we can define our mixing Hamiltonian on some qubits. As in the notebook on classical and quantum many-body physics, we had to define, for instance, an `IZ` operator to express $\mathbb{I}\otimes\sigma_1^Z$, that is, the $\sigma_1^Z$ operator acting only on qubit 1. We can achieve the same effect the following way (this time using the Pauli-X operator). The coefficient here means the strength of the transverse field at the given qubit. This operator will act trivially on all qubits, except the given one. Let's define the mixing Hamiltonian over two qubits:

In [3]:
Hm = [PauliTerm("X", i, -1.0) for i in range(n_qubits)]

As an example, we will minimize the Ising problem defined by the cost Hamiltonian $H_c=-\sigma^Z_1 \otimes \sigma^Z_2$, whose minimum is reached whenever $\sigma^Z_1 = \sigma^Z_2$ (for the states $|-1, -1\rangle$, $|11\rangle$ or any superposition of both)

In [4]:
J = np.array([[0,1],[0,0]]) # weight matrix of the Ising model. Only the coefficient (0,1) is non-zero.

Hc = []
for i in range(n_qubits):
    for j in range(n_qubits):
        Hc.append(PauliTerm("Z", i, -J[i, j]) * PauliTerm("Z", j, 1.0))

During the iterative procedure, we will need to compute $e^{-i \beta H_c}$ and $e^{-i \gamma H_m}$. Using the function `exponential_map` of PyQuil, we can build two functions that take respectively $\beta$ and $\gamma$ and return $e^{-i \beta H_c}$ and $e^{-i \gamma H_m}$

In [5]:
exp_Hm = []
exp_Hc = []
for term in Hm:
    exp_Hm.append(exponential_map(term))
for term in Hc:
    exp_Hc.append(exponential_map(term))

We set $p=2$ and initialize the $\gamma_i$ and $\nu_i$ parameters:

In [6]:
n_iter = 10 # number of iterations of the optimization procedure
p = 1
β = np.random.uniform(0, np.pi*2, p)
γ = np.random.uniform(0, np.pi*2, p)

The initial state is a uniform superposition of all the states $|q_1,...,q_n\rangle$. It can be created using Hadamard gates on all the qubits |0> of an new program.

In [7]:
initial_state = Program()
for i in range(n_qubits):
    initial_state += H(i)

To create the circuit, we need to compose the different unitary matrice given by `evolve`.

In [8]:
def create_circuit(β, γ):
    circuit = Program()
    circuit += initial_state
    for i in range(p):
        for term_exp_Hc in exp_Hc:
            circuit += term_exp_Hc(-β[i])
        for term_exp_Hm in exp_Hm:
            circuit += term_exp_Hm(-γ[i])

    return circuit

We now create a function `evaluate_circuit` that takes a single vector `beta_gamma` (the concatenation of $\beta$ and $\gamma$) and returns $\langle H_c \rangle = \langle \psi | H_c | \psi \rangle$ where $\psi$ is defined by the circuit created with the function above.

In [9]:
def evaluate_circuit(beta_gamma):
    β = beta_gamma[:p]
    γ = beta_gamma[p:]
    circuit = create_circuit(β, γ)
    return qvm.pauli_expectation(circuit, sum(Hc))

Finally, we optimize the angles:

In [10]:
qvm = api.QVMConnection(endpoint=fc.sync_endpoint, compiler_endpoint=fc.compiler_endpoint)

result = minimize(evaluate_circuit, np.concatenate([β, γ]), method='L-BFGS-B')
result

/home/artix41/miniconda3/envs/quantum-mooc/lib/python3.7/site-packages/pyquil/paulis.py:703: UserWarning: The term Z1Z0 will be combined with Z0Z1, but they have different orders of operations. This doesn't matter for QVM or wavefunction simulation but may be important when running on an actual device.
  .format(t.id(sort_ops=False), first_term.id(sort_ops=False)))
/home/artix41/miniconda3/envs/quantum-mooc/lib/python3.7/site-packages/scipy/optimize/optimize.py:670: ComplexWarning: Casting complex values to real discards the imaginary part
  grad[k] = (f(*((xk + d,) + args)) - f0) / d[k]
/home/artix41/miniconda3/envs/quantum-mooc/lib/python3.7/site-packages/scipy/optimize/lbfgsb.py:328: ComplexWarning: Casting complex values to real discards the imaginary part
  isave, dsave, maxls)


      fun: (-0.9999999999999977+0j)
 hess_inv: <2x2 LbfgsInvHessProduct with dtype=float64>
      jac: array([-0.,  0.])
  message: b'CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL'
     nfev: 51
      nit: 10
   status: 0
  success: True
        x: array([2.356, 1.178])

# Analysis of the results

We create a circuit using the optimal parameters found.

In [11]:
circuit = create_circuit(result['x'][:p], result['x'][p:])

We use the `statevector_simulator` backend in order to display the state created by the circuit.

In [12]:
wf_sim = api.WavefunctionSimulator(connection=fc)
state = wf_sim.wavefunction(circuit)
print(state)

(0.5+0.5j)|00> + (-1.09e-08-1.23e-08j)|01> + (-1.09e-08-1.23e-08j)|10> + (0.5+0.5j)|11>


We see that the state is approximately $(0.5 + 0.5i) \left( |00 \rangle + |11 \rangle \right) = e^{i \theta} \frac{1}{\sqrt{2}} \left( |00 \rangle + |11 \rangle \right)$, where $\theta$ is a phase factor that doesn't change the probabilities. It corresponds to a uniform superposition of the two solutions of the classicial problem: $(\sigma_1=1$, $\sigma_2=1)$ and $(\sigma_1=-1$, $\sigma_2=-1)$

Let's now try to evaluate the operators $\sigma^Z_1$ and $\sigma^Z_2$ independently:

In [13]:
print(qvm.pauli_expectation(circuit, PauliSum([sZ(0)])))
print(qvm.pauli_expectation(circuit, PauliSum([sZ(1)])))

0j
0j


We see that both are approximatively equal to zero. It's expected given the state we found above (both spins takes -1 and 1 half of the time). It corresponds to a typical quantum behavior where $\mathbb{E}[\sigma^Z_1 \sigma^Z_2] \neq \mathbb{E}[\sigma^Z_1] \mathbb{E}[\sigma^Z_2]$